# Implementación de algoritmo de detección de Maury y Revilla

Basado en el trabajo https://doi.org/10.1366/14-07834.

Carga de librerías externas, configuración de los parámetros a ser explorados durante la medición y nombre del archivo de donde se cargarán los datos espectrales.

In [ ]:
import math
import pywt
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
from random import sample
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

archivo_carga_espectros = '../resultados.ndjson'

params = [
    {
        'coef_adev': 1,
        'm': 1,
    },
    {
        'coef_adev': 2,
        'm': 1,
    },
    {
        'coef_adev': 3,
        'm': 1,
    },
    {
        'coef_adev': 4,
        'm': 1,
    },
    {
        'coef_adev': 5,
        'm': 1,
    },
    {
        'coef_adev': 1,
        'm': 2,
    },
    {
        'coef_adev': 2,
        'm': 2,
    },
    {
        'coef_adev': 3,
        'm': 2,
    },
    {
        'coef_adev': 4,
        'm': 2,
    },
    {
        'coef_adev': 5,
        'm': 2,
    }
]

Funciones de carga de espectros

In [ ]:
def deserializar_espectro(espectro):
    espectro['vector_base'] = np.array(espectro['vector_base'])
    espectro['espectro_base_muestra'] = np.array(espectro['espectro_base_muestra'])
    espectro['espectro_base_background'] = np.array(espectro['espectro_base_background'])
    espectro['baseline_muestra'] = np.array(espectro['baseline_muestra'])
    espectro['baseline_background'] = np.array(espectro['baseline_background'])
    espectro['muestra_con_baseline'] = np.array(espectro['muestra_con_baseline'])
    espectro['background_con_baseline'] = np.array(espectro['background_con_baseline'])
    espectro['muestra_combinado_base'] = np.array(espectro['muestra_combinado_base'])
    espectro['espectro_ruido_combinado'] = np.array(espectro['espectro_ruido_combinado'])
    espectro['espectro_ruido_background'] = np.array(espectro['espectro_ruido_background'])
    espectro['spikes_muestra'] = np.array(espectro['spikes_muestra'])
    espectro['spikes_background'] = np.array(espectro['spikes_background'])
    espectro['flag_spikes_muestra'] = np.array(espectro['flag_spikes_muestra'])
    espectro['flag_spikes_background'] = np.array(espectro['flag_spikes_background'])
    espectro['muestra_base_con_spikes'] = np.array(espectro['muestra_base_con_spikes'])
    espectro['y_muestra'] = np.array(espectro['y_muestra'])
    espectro['y_background'] = np.array(espectro['y_background'])

    return espectro

def cargar_espectros():
    with open(archivo_carga_espectros, 'r') as fp:
        data = []
        for line in fp:
            data.append(deserializar_espectro(json.loads(line)))
        fp.close()
        return data

espectros = cargar_espectros()

Función para calcular métricas clasificación de spikes.

In [ ]:
def calcular_metricas_clasificacion(valores_reales, predicciones):
    _predicciones = np.array(predicciones).flatten()
    _valores_reales = np.array(valores_reales).flatten()

    return precision_recall_fscore_support(_valores_reales, _predicciones), confusion_matrix(_valores_reales, _predicciones)

Implementación del algoritmo Maury-Revilla para detección de spikes, así como funciones auxiliares para recogida de datos y visualización de resultados.

In [ ]:
def algoritmo_maury_revilla(input, coef_adev = 3, K = 4, M = 2):
    # El primer paso es obtener la serie diferenciada
    serie_diferenciada = np.diff(input)
    ondicula = 'db' + str(K)

    # El segundo paso es aplicar un filtro por transformación ondicular
    # Como sugerido por los autores, el tamaño del filtro debe ser 2K=8, 
    # por lo que usamos la ondícula de Daubechies de orden 4 (2x4=8)
    # Adicionalmente los autores sugieren que para la mayoría de los casos hay que aplicar la
    # transformación a dos niveles (M=2)
    transformada_ondicular = pywt.wavedec(serie_diferenciada, ondicula, mode='periodic', level=M)
    
    # Para realizar el proceso inverso y reconstruir la señal original, se deben eliminar los filtros
    # de detalles, y dejar sólo los coeficientes de la "ondícula madre"
    for i in range(M + 1):
        if i > 0:
            transformada_ondicular[i] = np.zeros_like(transformada_ondicular[i])

    # Finalmente reconstruimos la serie, habiendo colocado en cero los coeficientes que no 
    # pertenecen a la "ondícula madre"
    serie_reconstruida = pywt.waverec(transformada_ondicular, ondicula, mode='periodic')

    # Calculamos la serie de valores residuales entre la serie diferenciada y la serie
    # diferenciada reconstruida. Eliminando el primer valor para que tengan igual dimensión
    residuales = np.subtract(serie_diferenciada, serie_reconstruida[1:])

    # Se procede luego a calcular el ADEV, siguiendo la fórmula indicada por los autores
    segunda_diferencia = np.diff(serie_diferenciada)
    suma_segundas_diferencias_cuadrado = np.sum(segunda_diferencia ** 2)
    adev = math.sqrt(suma_segundas_diferencias_cuadrado / (2 * (len(segunda_diferencia) - 2)))

    # Para las detecciones iniciales se considera el umbral
    detecciones = np.abs(residuales) > coef_adev * adev
    hay_detecciones_adicionales = True

    # Posteriormente deben realizarse las detecciones adicionales, que consideran
    # criterios lógicos y de cercanía de un punto del spike con otros puntos pertenecientes
    # a spikes. La idea es iterar hasta que ya no se detecten nuevos puntos de spikes, en
    # cuyo caso el algoritmo se detiene
    while hay_detecciones_adicionales:
        hay_detecciones_adicionales = False
        for i, deteccion in enumerate(detecciones):
            ventana_inferior = list(detecciones[0:i][-2:])
            ventana_superior = list(detecciones[i+1:][:2])

            ventana = np.array(ventana_inferior + ventana_superior)

            hay_detecciones = np.any(ventana)

            # Primero se busca si en una ventana de dos puntos alrededor del punto
            # que está siendo evaluado hay spikes, en cuyo caso el punto actual
            # se considera spike si el punto asociado en la serie diferenciada supera el
            # umbral
            if hay_detecciones and not detecciones[i]:
                nuevo_valor = abs(serie_diferenciada[i]) > coef_adev * adev
                if nuevo_valor:
                    detecciones[i] = nuevo_valor
                    hay_detecciones_adicionales = True

            ventana_inferior_reducida = list(detecciones[0:i][-1:])
            ventana_superior_reducida = list(detecciones[i+1:][:1])
            ventana_reducida = np.array(ventana_inferior_reducida + ventana_superior_reducida)

            rodeado_por_spikes = np.all(ventana_reducida)

            # Por otro lado, si hay un punto que está rodeado a ambos lados por puntos que son spikes
            # se le considerará también como parte del spike
            if rodeado_por_spikes and not detecciones[i]:
                detecciones[i] = True
                hay_detecciones_adicionales = True

    # Este algoritmo no puede detectar spikes en el primer valor de la serie,
    # nuestro interés es sólo la detección, por lo que nos interesa
    # preservar la precisión del algoritmo marcando como False el primer elemento, puesto
    # que esta es la elección más probable
    detecciones = np.insert(detecciones, 0, False)

    return detecciones, serie_diferenciada, serie_reconstruida, residuales, adev

def obtener_resultado_parametro_maury_revilla(espectros, etiquetas, coef_adev = 3, K = 4, M = 2):
    predicciones_maury_revilla = [algoritmo_maury_revilla(espectro, coef_adev, K, M)[0] for espectro in espectros]

    return predicciones_maury_revilla, calcular_metricas_clasificacion(etiquetas, predicciones_maury_revilla)

def visualizar_resultado_maury_revilla(vector_base, input, etiquetas, umbral_coef_adev = 3):
    detecciones, serie_diferenciada, serie_reconstruida, residuales, adev = algoritmo_maury_revilla(input, umbral_coef_adev)

    fig = plt.figure(figsize=[45, 45], constrained_layout=True)

    fig.suptitle("Muestra de resultados al aplicar el algoritmo Maury-Revilla", fontsize=24, fontweight='bold')

    subfigs = fig.subfigures(nrows=5, ncols=1)

    for row, subfig in enumerate(subfigs):
        (ax) = subfig.subplots(nrows=1, ncols=1)

        if row == 0:
            ax.set_title("Serie original", fontsize=18)
            ax.plot(vector_base, input)
        elif row == 1:
            ax.set_title("Serie diferenciada", fontsize=18)
            serie_diferenciada = np.insert(serie_diferenciada, 0, 0)
            ax.plot(vector_base, serie_diferenciada)
        elif row == 2:
            ax.set_title("Serie diferenciada reconstruida por transformación ondicular parcial", fontsize=18)
            ax.plot(vector_base, serie_reconstruida)
        elif row == 3:
            ax.set_title("Serie de valores residuales y umbral", fontsize=18)
            residuales = np.insert(residuales, 0, 0)
            ax.plot(vector_base, residuales)
            plt.axhline(y = umbral_coef_adev * adev, color = 'r', linestyle = '-')
            plt.axhline(y = -umbral_coef_adev * adev, color = 'r', linestyle = '-')
        elif row == 4:
            categorias = {
                'tp': {
                    'color': 'blue',
                    'label': 'Detecciones'
                },
                'fp': {
                    'color': 'black',
                    'label': 'Falsos positivos'
                },
                'fn': {
                    'color': 'red',
                    'label': 'Falsos negativos'
                }
            }

            detecciones_agrupadas = []

            for i, es_resultado in enumerate(detecciones):
                label = ''
                es_spike = etiquetas[i]

                if es_spike:
                    if es_resultado:
                        label = 'tp'
                    else:
                        label = 'fn'
                elif es_resultado:
                    label = 'fp'
                else:
                    label = 'tn'
                detecciones_agrupadas.append(label)

            detecciones_agrupadas = np.array(detecciones_agrupadas)

            ax.set_title("Serie con spikes detectados y tipos de detección", fontsize=18)
            ax.plot(vector_base, input)
            ax.scatter(vector_base[detecciones_agrupadas == 'tp'], input[detecciones_agrupadas == 'tp'], c=categorias['tp']['color'], label=categorias['tp']['label'], s=100)
            ax.scatter(vector_base[detecciones_agrupadas == 'fp'], input[detecciones_agrupadas == 'fp'], c=categorias['fp']['color'], label=categorias['fp']['label'], s=100)
            ax.scatter(vector_base[detecciones_agrupadas == 'fn'], input[detecciones_agrupadas == 'fn'], c=categorias['fn']['color'], label=categorias['fn']['label'], s=100)

            ax.legend(fontsize=28)

Separación de las curvas a emplear en la recogida de datos, así como las respectivas etiquetas que marcan la posición real de los spikes añadidos, del conjunto de datos espectrales cargado

In [ ]:
espectros_base_con_spikes_con_ruido = [espectro['muestra_base_con_spikes'] for espectro in espectros]
espectros_base_con_spikes_sin_ruido = [espectro['espectro_base_muestra'] + espectro['spikes_muestra'] for espectro in espectros]
etiquetas = [espectro['flag_spikes_muestra'] for espectro in espectros]

Recogida de datos y guardado de los resultados en un archivo.

In [ ]:
resultados_maury_revilla = []

for param in params:
    predicciones_maury_revilla_ruido, metricas_maury_revilla_ruido = obtener_resultado_parametro_maury_revilla(espectros_base_con_spikes_con_ruido, etiquetas, param['coef_adev'], param['m'])
    predicciones_maury_revilla_sin_ruido, metricas_maury_revilla_sin_ruido = obtener_resultado_parametro_maury_revilla(espectros_base_con_spikes_sin_ruido, etiquetas, param['coef_adev'], param['m'])

    for i in range(len(predicciones_maury_revilla_ruido)):
        resultados_maury_revilla.append({
            'etiquetas': etiquetas[i],
            'coef_adev': param['coef_adev'],
            'm': param['m'],
            'predicciones_ruido': predicciones_maury_revilla_ruido[i],
            'predicciones_sin_ruido': predicciones_maury_revilla_sin_ruido[i],
            'espectro_ruido': espectros_base_con_spikes_con_ruido[i],
            'espectro_sin_ruido': espectros_base_con_spikes_sin_ruido[i]
        })

with open('resultados_maury_revilla', 'wb') as fp:
    pickle.dump(resultados_maury_revilla, fp)

Toma de una muestra aleatoria para control de calidad

In [ ]:
indice_muestra = sample(list(range(len(espectros))), 1)[0]

Visualización del resultado de la aplicación del algoritmo Maury-Revilla a un espectro aleatorio al que no le fue añadido ruido aleatorio

In [ ]:
visualizar_resultado_maury_revilla(espectros[indice_muestra]['vector_base'], espectros_base_con_spikes_sin_ruido[indice_muestra], etiquetas[indice_muestra])

Visualización del resultado de la aplicación del algoritmo Maury-Revilla a un espectro aleatorio al que le fue añadido ruido aleatorio

In [ ]:
visualizar_resultado_maury_revilla(espectros[indice_muestra]['vector_base'], espectros_base_con_spikes_con_ruido[indice_muestra], etiquetas[indice_muestra])